# Top Routes

This notebook calculates and exports bar charts of:

- Top routes (unidirectional) over all time
- Top routes (bidirectional) over all time

In the unidirectional chart, LHR-JFK and JFK-LHR are distinct whereas in the bidirectional chart they are the same route.

In [1]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [2]:
# Top "N" routes to include
top_n = 20

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_unidirectional_chart(export_folder_path, data):
    # Generate the unidirectional bar chart
    fig = plt.figure(figsize=(12, 6))
    sns.barplot(data=data.head(top_n), x="Sightings", y="Route", palette="YlOrRd", hue="Route")
    plt.title(f"Top {top_n} Most Common Aircraft Routes (Unidirectional)")
    plt.xlabel("Number of Sightings")
    plt.ylabel("Route")
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"top-routes-unidirectional", "png")

    # Close the plot
    plt.close(fig)


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_bidirectional_chart(export_folder_path, data):
    # Generate the bidirectional bar chart
    fig = plt.figure(figsize=(12, 6))
    sns.barplot(data=normalised_counts.head(top_n), x="Sightings", y="Normalised_Route", palette="YlOrRd", hue="Normalised_Route")
    plt.title(f"Top {top_n} Most Common Aircraft Routes (Bidirectional)")
    plt.xlabel("Number of Sightings")
    plt.ylabel("Route")
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"top-routes-bidirectional", "png")

    # Close the plot
    plt.close(fig)


In [ ]:
import pandas as pd
import warnings

# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Remove entries where either the point of embarkation or destination aren"t available
    warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
    df_cleaned = df[(df["Embarkation"] != "N/A") & (df["Destination"] != "N/A")]

    # Combine embarkation and destination into a single route column
    df_cleaned["Route"] = df_cleaned["Embarkation"].str.strip() + "-" + df_cleaned["Destination"].str.strip()

    # Count unidirectional routes. In this scheme, LHR-JFK is different from JFK-LHR
    route_counts = df_cleaned["Route"].value_counts().reset_index()
    route_counts.columns = ["Route", "Sightings"]

    # Define a function to normalise a route
    def normalize_route(row):
        airports = sorted([row["Embarkation"].strip(), row["Destination"].strip()])
        return f"{airports[0]}-{airports[1]}"

    # Use it to add a normalised route column to the data
    df_cleaned["Normalised_Route"] = df_cleaned.apply(normalize_route, axis=1)

    # Count normalized route frequencies
    normalised_counts = df_cleaned["Normalised_Route"].value_counts().reset_index()
    normalised_counts.columns = ["Normalised_Route", "Sightings"]

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "top-routes-unidirectional.xlsx", {
        "Unidirectional": route_counts
    })
    export_to_spreadsheet(export_folder_path, "top-routes-bidirectional.xlsx", {
        "Bidirectional": normalised_counts,
    })

    # Export the charts
    export_unidirectional_chart(export_folder_path, route_counts)
    export_bidirectional_chart(export_folder_path, normalised_counts)

except ValueError:
    print(f"WARNING: No data found")